# Анализ влияния товарного ассортимента на выручку ТТ ритейла

__Задача__

Исследовать влияние товарного ассортимента на выручку магазина в целях улучшения алгоритма подбора товарного ассортимента.

### Подготовка данных

In [1]:
import pandas as pd
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')

Файл информации о ТТ:

In [2]:
zao = pd.read_csv('zao_info.csv', sep=';', usecols=[0, 5])

Файлы чеков за апрель-май 2021 и апрель-май 2022

In [ ]:
april_may21_chek = pd.read_csv('ch_zao_april-may21.csv', sep=';', usecols=[1, 4, 5], decimal=',')
april22_chek = pd.read_csv('ch_zao_april.csv', sep=';', usecols=[1, 4, 5], decimal=',')
may22_chek = pd.read_csv('ch_zao_may.csv', sep=';', usecols=[1, 4, 5], decimal=',')

In [ ]:
april_may21_chek['CloseDate'] = pd.to_datetime(april_may21_chek['CloseDate'], format='%Y %m %d %H:%M:%S.%f').dt.round('S').dt.date
april22_chek['CloseDate'] = pd.to_datetime(april22_chek['CloseDate'], format='%Y %m %d %H:%M:%S.%f').dt.round('S').dt.date
may22_chek['CloseDate'] = pd.to_datetime(may22_chek['CloseDate'], format='%Y %m %d %H:%M:%S.%f').dt.round('S').dt.date

In [ ]:
april_may21_chek = april_may21_chek.rename(columns={'id_TT': 'id_tt', 'CloseDate': 'date_ch'})
april22_chek = april22_chek.rename(columns={'id_TT': 'id_tt', 'CloseDate': 'date_ch'})
may22_chek= may22_chek.rename(columns={'id_TT': 'id_tt', 'CloseDate': 'date_ch'})

Файлы продаж за май 2021 и 2022 годов:

In [ ]:
may21 = pd.read_csv('zao_mayl21.csv', sep=';', usecols=[1, 5, 6, 7, 8])
may22 = pd.read_csv('1_may.csv', sep=';', usecols=[0, 1, 2, 6, 7])

In [7]:
may21= may21.rename(columns={'id_tt_cl': 'id_tt', 'id_tov_cl': 'id_tov'})
may22= may22.rename(columns={'id_tt_cl': 'id_tt', 'id_tov_cl': 'id_tov'})

In [8]:
may21 = may21[['CheckUID', 'id_tt', 'id_tov', 'BaseSum', 'date_ch']]

Файл SKU:

In [9]:
sku = pd.read_csv('tov_info.csv', sep=';', usecols=[0, 1, 4, 5, 6, 7])

In [10]:
assort = sku[['id_tov', 'Name_tov', 'id_group', 'Name_gr', 'id_group2', 'Name_gr2']]

## Список ТТ для исследования:

Объединю таблицы:

In [11]:
chek = pd.concat([april_may21_chek, april22_chek, may22_chek], ignore_index=True)

Изменю тип данных:

In [12]:
chek['date_ch'] = pd.to_datetime(chek['date_ch'])

In [13]:
chek['BaseSum'] = chek['BaseSum'].astype(int)

Вычислю помесячную выручку для каждого магазина

In [14]:
revenue = chek.groupby(['id_tt', pd.Grouper(key='date_ch', freq='M')])['BaseSum'].sum().reset_index()

Создам таблицу помесячной выручки:

In [15]:
revenue_pivot = pd.pivot_table(revenue,
               index=['id_tt'],
               columns=['date_ch'],
               values=['BaseSum'],
               )

In [16]:
revenue_pivot.reset_index(inplace=True)

Переименую столбцы:

In [17]:
revenue_pivot.columns = ['id_tt', 'апрель_2021', 'май_2021', 'апрель_2022', 'май_2022']

Объединю таблицы zao и revenue_pivot:

In [18]:
revenue = pd.merge(zao, revenue_pivot, how='left', on='id_tt')

Удалю строки с пропущенными значениями:

In [19]:
revenue.dropna(axis=0, inplace=True)

In [20]:
revenue.isna().sum().sum()

0

Посчитаю среднемесячную выручку за апрель-май 2021 и 2022 года.

In [21]:
revenue['2021(4-5)'] = ((revenue['апрель_2021'] + revenue['май_2021']) / 2).astype(int)
revenue['2022(4-5)'] = ((revenue['апрель_2022'] + revenue['май_2022']) / 2).astype(int)
revenue = revenue[['id_tt', 'ploshad', '2021(4-5)', '2022(4-5)']]

Посчитаю изменение выручки год к году:

In [22]:
revenue['Изменение, %'] = ((revenue['2022(4-5)']  - revenue['2021(4-5)']) / revenue['2021(4-5)']*100).round(2)

Список хороших магазинов:

In [23]:
GOOD_tt = revenue[revenue['Изменение, %'] >= 0]

In [24]:
GOOD_tt = GOOD_tt['id_tt']

Список плохих магазинов:

In [26]:
BAD_tt = revenue[revenue['Изменение, %'] < 0]

In [27]:
BAD_tt = BAD_tt['id_tt']

Выведу ТОП и антиТОП ТТ по изменению выручки:

In [29]:
tt_GOOD = revenue.sort_values(by='Изменение, %', ascending=False).head(5)
tt_BAD = revenue.sort_values(by='Изменение, %', ascending=False).tail(5)

## Количественный анализ

__Количественное сравнение ассортимента по всем ТТ:__

Создам таблицы для исследования:

In [32]:
tt_may21 = pd.merge(may21, sku, how='left', on='id_tov')
tt_may22 = pd.merge(may22, sku, how='left', on='id_tov')

Подготовлю таблицу с данными за май 2021 года:

In [33]:
tt_pivot_may21 = tt_may21.pivot_table(index='id_tt', values=['id_tov', 'BaseSum', 'CheckUID', 'id_group', 'id_group2'], aggfunc={'id_tov':'nunique', 'BaseSum':'sum', 'CheckUID':'nunique', 'id_group2':'nunique', 'id_group':'nunique'})
tt_pivot_may21 = tt_pivot_may21.reset_index().astype(int)

Переименую столбцы:

In [34]:
tt_pivot_may21.columns = ['id_tt', '2021-05_revenue', '2021-05_chec_count', '2021-05_gr_count', '2021-05_gr2_count', '2021-05_tov_count']

Объединю таблицы zao и tt_pivot_may21

In [35]:
tt_pivot_may21 = pd.merge(zao, tt_pivot_may21, how='left', on='id_tt')

Удалю строки с пропущенными значениями:

In [36]:
tt_pivot_may21.dropna(axis=0, inplace=True)

Изменю тип данных:

In [37]:
tt_pivot_may21[['id_tt', '2021-05_revenue', '2021-05_chec_count', '2021-05_gr_count', '2021-05_gr2_count', '2021-05_tov_count']] = tt_pivot_may21[['id_tt', '2021-05_revenue', '2021-05_chec_count', '2021-05_gr_count', '2021-05_gr2_count', '2021-05_tov_count']].astype(int)

Подготовлю таблицу с данными за май 2022 года:

In [39]:
tt_pivot_may22 = tt_may22.pivot_table(index='id_tt', values=['id_tov', 'BaseSum', 'CheckUID', 'id_group', 'id_group2'], aggfunc={'id_tov':'nunique', 'BaseSum':'sum', 'CheckUID':'nunique', 'id_group2':'nunique', 'id_group':'nunique'})
tt_pivot_may22 = tt_pivot_may22.reset_index().astype(int).sort_values('BaseSum')

Переименую столбцы:

In [40]:
tt_pivot_may22.columns = ['id_tt', '2022-05_revenue', '2022-05_chec_count', '2022-05_gr_count', '2022-05_gr2_count', '2022-05_tov_count']

Объединю таблицы tt_pivot_may21 и tt_pivot_may22.columns

In [41]:
tt_merge_may21_22 = pd.merge(tt_pivot_may21, tt_pivot_may22, how='left', on='id_tt')

Добавлю столбец относительного изменения товарного ассортимента по количеству за год май к маю:

In [43]:
tt_merge_may21_22['change_tov,%'] = ((tt_merge_may21_22['2022-05_tov_count'] - tt_merge_may21_22['2021-05_tov_count']) / tt_merge_may21_22['2021-05_tov_count']*100).round(2)

Посчитаю изменение выручки за год май к маю:

In [44]:
tt_merge_may21_22['revenue_change,%'] = ((tt_merge_may21_22['2022-05_revenue'] - tt_merge_may21_22['2021-05_revenue']) / tt_merge_may21_22['2021-05_revenue']*100).round(2)

Посмотрю корреляции

In [46]:
print('Корреляция изменения товарного ассортимента с изменением выручки за год  май к маю:', (tt_merge_may21_22['revenue_change,%'].corr(tt_merge_may21_22['change_tov,%'])).round(2))
print('Корреляция товарного ассортимента с выручкой за май 2021 года:', (tt_merge_may21_22['2021-05_revenue'].corr(tt_merge_may21_22['2021-05_tov_count'])).round(2))
print('Корреляция товарного ассортимента с выручкой за май 2022 года:', (tt_merge_may21_22['2022-05_revenue'].corr(tt_merge_may21_22['2022-05_tov_count'])).round(2))
print('Среднее изменение товарного ассортимента за год  май к маю по всем ТТ:', (tt_merge_may21_22['change_tov,%'].median()).round(2))
print('Среднее изменение выручки за год  май к маю по всем ТТ:', (tt_merge_may21_22['revenue_change,%'].median()).round(2))


Корреляция изменения товарного ассортимента с изменением выручки за год  май к маю: 0.67
Корреляция товарного ассортимента с выручкой за май 2021 года: 0.75
Корреляция товарного ассортимента с выручкой за май 2022 года: 0.8
Среднее изменение товарного ассортимента за год  май к маю по всем ТТ: 7.32
Среднее изменение выручки за год  май к маю по всем ТТ: -6.36


#### Вывод:
Наблюдается снижение продаж за год май к маю при одновременном увеличении товарного ассортимента.

## Качественный анализ:

__Май 2021 года:__

In [47]:
tt_may21_tov = tt_may21.groupby('Name_tov').agg(may21_tov_count=('Name_tov', 'count'),
                                                may21_revenue=('BaseSum', 'sum')
                                                ).reset_index()

Топ товара по количеству:

In [ ]:
tt_may21_tov['may21_revenue'] = tt_may21_tov['may21_revenue'].astype(int)
tt_may21_tov.sort_values(by='may21_tov_count', ascending=False).head(10)

Топ товара по объёму продаж:

In [ ]:
tt_may21_tov.sort_values(by='may21_revenue', ascending=False).head(10)

Топ товарной номенклатуры по количеству:

In [50]:
tt_may21_gr2 = tt_may21.groupby('Name_gr2').agg(may21_gr2_count=('Name_gr2', 'count'),
                                                may21_revenue=('BaseSum', 'sum')
                                                ).reset_index()

In [ ]:
tt_may21_gr2['may21_revenue'] = tt_may21_gr2['may21_revenue'].astype(int)
tt_may21_gr2.sort_values(by='may21_gr2_count', ascending=False).head(10)

Топ товарной номенклатуры по объёму продаж:

In [ ]:
tt_may21_gr2.sort_values(by='may21_revenue', ascending=False).head(10)

__Май 2022 года__

In [53]:
tt_may22_tov = tt_may22.groupby('Name_tov').agg(may22_tov_count=('Name_tov', 'count'),
                                                may22_revenue=('BaseSum', 'sum')
                                                ).reset_index()

Топ товара по количеству:

In [ ]:
tt_may22_tov['may22_revenue'] = tt_may22_tov['may22_revenue'].astype(int)
tt_may22_tov.sort_values(by='may22_tov_count', ascending=False).head(10)

Топ товара по объёму продаж:

In [ ]:
tt_may22_tov.sort_values(by='may22_revenue', ascending=False).head(10)

Топ товарной номенклатуры по количеству:

In [56]:
tt_may22_gr2 = tt_may22.groupby('Name_gr2').agg(may22_gr2_count=('Name_gr2', 'count'),
                                                may22_revenue=('BaseSum', 'sum')
                                                ).reset_index()

In [ ]:
tt_may22_gr2['may22_revenue'] = tt_may22_gr2['may22_revenue'].astype(int)
tt_may22_gr2.sort_values(by='may22_gr2_count', ascending=False).head(10)

Топ товарной номенклатуры по объёму продаж:

In [ ]:
tt_may22_gr2.sort_values(by='may22_revenue', ascending=False).head(10)

#### Вывод

Заметно, что за год предпочтения покупателей сдвинулись в сторону уменьшения покупок овощей и зелени.

## Исследование хороших и плохих ТТ.

### Хорошие ТТ:

In [59]:
tt_GOOD = tt_GOOD['id_tt']

In [60]:
tt_GOOD_may22 = pd.merge(tt_GOOD, may22, how='left', on='id_tt')

In [61]:
tt_GOOD_may22 = pd.merge(tt_GOOD_may22, sku, how='left', on='id_tov')

In [ ]:
tt_GOOD_pivot_may22 = tt_GOOD_may22.pivot_table(index='id_tt', values=['Name_tov', 'CheckUID', 'id_group', 'Name_gr2', 'BaseSum'], aggfunc={'Name_tov':'nunique', 'CheckUID':'nunique', 'Name_gr2':'nunique', 'id_group':'nunique', 'BaseSum':'sum'})
tt_GOOD_pivot_may22.reset_index().astype(int).sort_values('BaseSum')

In [ ]:
print('Среднее количество товарного ассортимента ТТ_хорошие за май 2022 года - ', tt_GOOD_pivot_may22['Name_tov'].median())

Товарный ассортимент:

In [ ]:
tt_GOOD_may22['Name_tov'].value_counts(normalize=True).head(10)

Товарная номенклатура

In [ ]:
tt_GOOD_may22['Name_gr2'].value_counts(normalize=True).head(10)

### Плохие ТТ

Подготовлю таблицы

In [66]:
tt_BAD = tt_BAD['id_tt']

In [67]:
tt_BAD_may22 = pd.merge(tt_BAD, may22, how='left', on='id_tt')

In [68]:
tt_BAD_may22 = pd.merge(tt_BAD_may22, sku, how='left', on='id_tov')

In [ ]:
tt_BAD_pivot_may22 = tt_BAD_may22.pivot_table(index='id_tt', values=['Name_tov', 'CheckUID', 'id_group', 'Name_gr2', 'BaseSum'], aggfunc={'Name_tov':'nunique', 'CheckUID':'nunique', 'Name_gr2':'nunique', 'id_group':'nunique', 'BaseSum':'sum'})
tt_BAD_pivot_may22.reset_index().astype(int).sort_values('BaseSum')

In [70]:
print('Среднее количество товарного ассортимента ТТ_плохие за май 2022 года - ', tt_BAD_pivot_may22['Name_tov'].median())

Среднее количество товарного ассортимента ТТ_плохие за май 2022 года -  3193.0


Товарный ассортимент

In [ ]:
tt_BAD_may22['Name_tov'].value_counts(normalize=True).head(10)

Товарная номенклатура:

In [ ]:
tt_BAD_may22['Name_gr2'].value_counts(normalize=True).head(10)

#### Вывод:

В плохих магазинах по сравнению с хорошими, товарный ассортимент не учитывает предпочтения покупателей как по количеству(3193 против 3702), так и по качеству - не хватает овощей и зелени.

## Статистическая проверка гипотезы

__Гипотеза:__
Товарный ассортимент по количеству влияет на выручку.<br>
 Для сравнения выберу хорошие и плохие ТТ по показателю изменение товарного ассортимента за год май к маю.<br>
 Сформулирую гипотезы:<br>
 1. Нулевая гипотеза: статистически значимых различий между выборками нет.<br>
 2. Альтернативная гипотеза: статистически значимые отличия между выборками есть.<br>

In [73]:
sampleA = tt_merge_may21_22.query('id_tt in @GOOD_tt')['change_tov,%']
sampleB = tt_merge_may21_22.query('id_tt in @BAD_tt')['change_tov,%']

In [74]:
alpha = 0.05

results = stats.ttest_ind(sampleA, sampleA, equal_var = False)

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

p-значение: 1.0
Не получилось отвергнуть нулевую гипотезу


Вывод: Не получилось отвергнуть нулевую гипотезу, значит товарный ассортимент по количеству не влияет на выручку. Тест проводился на достаточно малой выборке, но вполне вероятно и на большой выборке результат будет тот же.